In [1]:
import pandas as pd
from datetime import datetime
import numpy as np
import glob
import os


## Clean up overdose data

In [2]:
orig_cols_to_keep = ["REPORTER_DEA_NO", "REPORTER_STATE", "REPORTER_ZIP", "REPORTER_COUNTY", "DRUG_CODE", "TRANSACTION_CODE", "DRUG_NAME", "QUANTITY", "TRANSACTION_DATE", "CALC_BASE_WT_IN_GM", "DOSAGE_UNIT",
                "TRANSACTION_ID", "Product_Name", "Ingredient_Name", "Measure", "dos_str"]
# keeping more columns than is probably necessary
# just want to make sure we have everything we need, as well as have the opportunity to catch any further filtering we miss at first
pd.set_option('display.max_columns', None)


In [3]:
def find_year(TRANSACTION_DATE):
    """
    Args:
        TRANSACTION_DATE (str): date in format MMDDYYYY

    Returns:
        int: year
    """
    TRANSACTION_DATE = str(TRANSACTION_DATE)
    
    return int(TRANSACTION_DATE[-4:])

# quick test 
print(f"testing the find_year function: {find_year(12202019)}")


def find_month(TRANSACTION_DATE):
    """
    Args:
        TRANSACTION_DATE (str): date in format MMDDYYYY

    Returns:
        int: month
    """
    TRANSACTION_DATE = str(TRANSACTION_DATE)
    
    return int(TRANSACTION_DATE[:2])

# quick test 
print(f"testing the find_month function: {find_month(12202019)}")

testing the find_year function: 2019
testing the find_month function: 12


In [4]:
# to load in the data, we need to truncate the amount of columns we use as well as the states
cols_to_keep = ["REPORTER_DEA_NO", "REPORTER_STATE", "REPORTER_ZIP", "REPORTER_COUNTY", "DRUG_CODE", "TRANSACTION_CODE", "DRUG_NAME", "QUANTITY", "TRANSACTION_DATE", "Product_Name"]

# we know we need Florida, Texas, and Washington
states = ["FL", "TX", "WA"]
# since we are normalizing based on population, I think we should pick states that are regionally close to our target states
# we can change this later as a group, but I have these selected below:

# Florida comparison states: Georgia, Alabama, Missisippi, South Carolina, Tennessee
fl_states = ["GA", "AL", "MS", "SC", "TN"]

# Texas comparison states: Oklahoma, Louisiana, New Mexico, Arkansas, Kansas
tx_states = ["OK", "LA", "NM", "AR", "KS"]

# Washington comparison states: Oregon, Idaho, Montana, Nevada, Wyoming
wa_states = ["OR", "ID", "MT", "NV", "WY"]

# create list of all states to use
variable_states = []
variable_states.extend(fl_states)
variable_states.extend(tx_states)
variable_states.extend(wa_states)

# append variable states to our original list
states.extend(variable_states)


In [5]:
# now, load in our data as an iterator so we can load in chunks
it = pd.read_csv("00_source_data/arcos_all_washpost.tsv", chunksize=500_000, sep='\t', usecols = cols_to_keep) # may have to change chunksize depending on your computer's memory

# init empty dataframe
df = pd.DataFrame()

for chunk in it:
    # extract year out of date column
    chunk["year"] = chunk["TRANSACTION_DATE"].apply(lambda x: find_year(x))
    chunk["month"] = chunk["TRANSACTION_DATE"].apply(lambda x: find_month(x))

    # ensure we're working in the correct date range
    filtered_chunk = chunk[chunk["year"] > 2002]
    filtered_chunk = filtered_chunk[filtered_chunk["year"] < 2016]

    # filter out the states we want
    filtered_chunk = filtered_chunk[filtered_chunk["REPORTER_STATE"].isin(states)]

    df = df.append(filtered_chunk)
    break
df

df_overdoses = df.copy() # keep a copy of this df for later filtering

In [6]:
df_overdoses.head()

,REPORTER_DEA_NO,REPORTER_STATE,REPORTER_ZIP,REPORTER_COUNTY,TRANSACTION_CODE,DRUG_CODE,DRUG_NAME,QUANTITY,TRANSACTION_DATE,Product_Name,year,month
6224,PB0167127,GA,31793,TIFT,S,9193,HYDROCODONE,2.0,11232009,HYDROCODONE BIT/IBUPROFEN 7.5MG/200M,2009,11
6225,PB0167127,GA,31793,TIFT,S,9193,HYDROCODONE,2.0,12172012,HYDROCODONE BIT./ACETAMINOPHEN TABS.,2012,12
6226,PB0167127,GA,31793,TIFT,S,9193,HYDROCODONE,1.0,2192007,HYDROCODONE/IBUPROFEN 7.5MG/200MG TA,2007,21
6227,PB0167127,GA,31793,TIFT,S,9193,HYDROCODONE,1.0,6102011,HYDROCODONE BIT/ACETA 10MG/325MG USP,2011,61
6228,PB0167127,GA,31793,TIFT,S,9193,HYDROCODONE,1.0,6022008,HYDROCODONE BIT/ACETA 7.5MG/325MG US,2008,60


In [8]:
# now that we have our columns and states filtered, let's export this as a csv and store the (much smaller) result on github

df_overdoses.to_csv("05_cleaned_data/arcos_all_washpost_clean.csv", index=False)

## Clean up cause of death data

In [9]:
path = r'00_source_data/cause_of_death' # use your path
filenames = glob.glob(path + "/*.txt")

df = pd.DataFrame()

for f in filenames:
    temp = pd.read_csv(f, index_col=None, header=0, sep='\t')
    # we're getting some extraneous notes at the bottom - let's just drop for now
    temp.dropna(subset={'County'}, inplace=True)
    
    df = pd.concat([df, temp], axis=0, ignore_index=True)

In [10]:
df

,Notes,County,County Code,Year,Year Code,Drug/Alcohol Induced Cause,Drug/Alcohol Induced Cause Code,Deaths
0,NaN,"Autauga County, AL",1001.0,2003.0,2003.0,All other non-drug and non-alcohol causes,O9,397.0
1,NaN,"Baldwin County, AL",1003.0,2003.0,2003.0,Drug poisonings (overdose) Unintentional (X40-...,D1,10.0
2,NaN,"Baldwin County, AL",1003.0,2003.0,2003.0,All other alcohol-induced causes,A9,14.0
3,NaN,"Baldwin County, AL",1003.0,2003.0,2003.0,All other non-drug and non-alcohol causes,O9,1479.0
4,NaN,"Barbour County, AL",1005.0,2003.0,2003.0,All other non-drug and non-alcohol causes,O9,287.0
...,...,...,...,...,...,...,...,...
57236,NaN,"Sweetwater County, WY",56037.0,2015.0,2015.0,All other non-drug and non-alcohol causes,O9,251
57237,NaN,"Teton County, WY",56039.0,2015.0,2015.0,All other non-drug and non-alcohol causes,O9,95
57238,NaN,"Uinta County, WY",56041.0,2015.0,2015.0,All other non-drug and non-alcohol causes,O9,142
57239,NaN,"Washakie County, WY",56043.0,2015.0,2015.0,All other non-drug and non-alcohol causes,O9,81


In [11]:
# helper functions to separate county and state

def abtract_state(county):
    """
    Args:
        county (str): county name

    Returns:
        str: state
    """
    return county.split(", ")[1]



def abstract_county(county):
    """
    Args:
        county (str): county name

    Returns:
        str: county
    """
    return county.split(", ")[0]


df["State"] = df.apply(lambda x: abtract_state(x["County"]), axis=1)
df["County"] = df.apply(lambda x: abstract_county(x["County"]), axis=1)

df.drop(columns={"Notes"}, inplace=True)

df_cause_of_death = df.copy() # keep a copy of this df for later filtering

In [12]:
df_cause_of_death.head()

,County,County Code,Year,Year Code,Drug/Alcohol Induced Cause,Drug/Alcohol Induced Cause Code,Deaths,State
0,Autauga County,1001.0,2003.0,2003.0,All other non-drug and non-alcohol causes,O9,397.0,AL
1,Baldwin County,1003.0,2003.0,2003.0,Drug poisonings (overdose) Unintentional (X40-...,D1,10.0,AL
2,Baldwin County,1003.0,2003.0,2003.0,All other alcohol-induced causes,A9,14.0,AL
3,Baldwin County,1003.0,2003.0,2003.0,All other non-drug and non-alcohol causes,O9,1479.0,AL
4,Barbour County,1005.0,2003.0,2003.0,All other non-drug and non-alcohol causes,O9,287.0,AL


In [13]:
df_cause_of_death.to_csv("05_cleaned_data/cause_of_death_clean.csv", index=False)

## Final 3 datasets

We should have: (UNSURE IF WE SHOULD EXTEND DATE RANGES, CURRENTLY 3 YEARS BEFORE AND AFTER POLICY IMPLEMENTATION)

- Florida and Georgia 2007 - 2013
- Texas and Oklahoma 2004 - 2010
- Washington and Oregon 2009 - 2015

### Drug overdose - broken down by state

In [14]:
# Florida and Georgia

overdoses_fl = df_overdoses.copy()
overdoses_tx = df_overdoses.copy()
overdoses_wa = df_overdoses.copy()

overdoses_fl_ga = overdoses_fl[(overdoses_fl["REPORTER_STATE"] == "FL") | (overdoses_fl["REPORTER_STATE"].isin(fl_states))]
overdoses_tx_ok = overdoses_tx[(overdoses_tx["REPORTER_STATE"] == "TX") | (overdoses_tx["REPORTER_STATE"]).isin(tx_states)]
overdoses_wa_or = overdoses_wa[(overdoses_wa["REPORTER_STATE"] == "WA") | (overdoses_wa["REPORTER_STATE"]).isin(wa_states)]



# filter appropriate years
fl_start = 2007
fl_end = 2013

tx_start = 2004
tx_end = 2010

wa_start = 2009
wa_end = 2015


overdoses_fl = overdoses_fl[(overdoses_fl_ga["year"] >= fl_start) & (overdoses_fl_ga["year"] <= fl_end)]
overdoses_tx = overdoses_tx[(overdoses_tx_ok["year"] >= tx_start) & (overdoses_tx_ok["year"] <= tx_end)]
overdoses_wa = overdoses_wa[(overdoses_wa_or["year"] >= wa_start) & (overdoses_wa_or["year"] <= wa_end)]


C:\Users\abzdel\AppData\Local\Temp/ipykernel_18076/1522061624.py:24: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  overdoses_fl = overdoses_fl[(overdoses_fl_ga["year"] >= fl_start) & (overdoses_fl_ga["year"] <= fl_end)]


IndexingError: Unalignable boolean Series provided as indexer (index of the boolean Series and of the indexed object do not match).

### Cause of death - broken down by state

In [ ]:
# ensure states are now in their full form to match the cause of death data
fl_states = ["Georgia", "Alabama", "Missisippi", "South Carolina", "Tennessee"]

tx_states = ["Oklahoma", "Louisiana", "New Mexico", "Arkansas", "Kansas"]

wa_states = ["Oregon", "Idaho", "Montana", "Nevada", "Wyoming"]

In [ ]:
deaths_fl = df_cause_of_death.copy()
deaths_tx = df_cause_of_death.copy()
deaths_wa = df_cause_of_death.copy()

deaths_fl = deaths_fl[(deaths_fl["State"] == "Florida") | (deaths_fl["State"].isin(fl_states))]
deaths_tx = deaths_tx[(deaths_tx["State"] == "Texas") | (deaths_tx["State"].isin(tx_states))]
deaths_wa = deaths_wa[(deaths_wa["State"] == "Washington") | (deaths_wa["State"].isin(wa_states))]

deaths_fl = deaths_fl[(deaths_fl["Year"] >= fl_start) & (deaths_fl["Year"] <= fl_end)]
deaths_tx = deaths_tx[(deaths_tx["Year"] >= tx_start) & (deaths_tx["Year"] <= tx_end)]  
deaths_wa = deaths_wa[(deaths_wa["Year"] >= wa_start) & (deaths_wa["Year"] <= wa_end)]

### export all to csv

In [ ]:
overdoses_fl.to_csv("05_cleaned_data/overdoses_fl.csv", index=False)
overdoses_tx.to_csv("05_cleaned_data/overdoses_tx.csv", index=False)
overdoses_wa.to_csv("05_cleaned_data/overdoses_wa.csv", index=False)

deaths_fl.to_csv("05_cleaned_data/deaths_fl.csv", index=False)
deaths_tx.to_csv("05_cleaned_data/deaths_tx.csv", index=False)
deaths_wa.to_csv("05_cleaned_data/deaths_wa.csv", index=False)

## Notes for the group

- need to normalize over population once we get this data
- may need to filter out a couple more columns - haven't done this yet as I don't want to accidentally delete something we need
- overdose data is only broken down by year unless i messed something up - not really sure how to handle this